# 从huggingface加载数据集

In [ ]:
from datasets import load_dataset
# 数据集地址 https://huggingface.co/datasets/naklecha/minecraft-question-answer-700k
# 指定数据集的URL
# 加载数据集并添加来源URL
dataset = load_dataset("naklecha/minecraft-question-answer-700k",
                       split="train" )

## 如果出现网络问题，请手动下载数据集

# 从本地json读取数据集

In [1]:
import json

# 打开本地的JSON文件
with open('./dataset/minecraft-question-answer-700k.json', 'r') as file:
    # 读取文件内容
    data = json.load(file)

# 处理JSON数据
# 例如，打印JSON数据
# [{},{},{}]

print('数量：',len(data))

数量： 694814


In [2]:
from datasets import Dataset
my_data = {"train": data}
dataset = Dataset.from_dict(my_data)

In [3]:
# 查看数据集的前2条，每一条由 question answer source 三个字段构成
dataset[0:2]

{'train': [{'answer': 'Saturation is the first statistic to decrease when a player performs energy-intensive actions, and it must be completely depleted before the visible hunger meter begins decreasing.',
   'question': 'What is the first statistic to decrease when a player performs energy-intensive actions in Minecraft?',
   'source': 'https://minecraft.wiki/w/Food#Nourishment_value'},
  {'answer': 'Eating cake is distinct from other foods, as it must be placed and then right-clicked on to consume, whereas other foods can be eaten directly by the player. Additionally, cake has 7 edible slices, which become thinner as each slice is removed, whereas other foods typically restore a set amount of hunger and saturation points without any slice-based consumption mechanism.',
   'question': 'How does the game handle the consumption of cake when compared to eating other types of food?',
   'source': 'https://minecraft.wiki/w/Food#Nourishment_value'}]}

# 安装openai的库，用来处理数据集

In [29]:
# 安装openai的库，用来处理数据集
!pip install openai

Looking in indexes: https://pypi.python.org/simple, https://pypi.ngc.nvidia.com
                                              0.0/311.6 kB ? eta -:--:--
     ---                                     30.7/311.6 kB 1.4 MB/s eta 0:00:01
     ---                                     30.7/311.6 kB 1.4 MB/s eta 0:00:01
     -------                               61.4/311.6 kB 409.6 kB/s eta 0:00:01
     -------------                        112.6/311.6 kB 656.4 kB/s eta 0:00:01
     --------------                       122.9/311.6 kB 554.9 kB/s eta 0:00:01
     ----------------------------         245.8/311.6 kB 888.8 kB/s eta 0:00:01
     -------------------------------------- 311.6/311.6 kB 1.1 MB/s eta 0:00:00
                                              0.0/409.3 kB ? eta -:--:--
     -----------------------                256.0/409.3 kB 5.2 MB/s eta 0:00:01
     ------------------------------------   389.1/409.3 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 409.3/409.


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 使用LLM来处理数据-支持本地LLM、OpenAI、Azure

In [5]:
import openai
from openai import AzureOpenAI
import random

def read_key_from_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
        return text
        
def get_client(api_key,base_url,deployment,is_azure=False):
    client=None
    if is_azure:  
        #gets the API Key from environment variable AZURE_OPENAI_API_KEY
        client = AzureOpenAI(
            # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
            api_version="2023-07-01-preview",
            # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
            azure_endpoint=base_url,#"https://example-endpoint.openai.azure.com"
            azure_ad_token=api_key,
            azure_deployment=deployment,
            default_headers = {'api-key': api_key }
        )
    else:
        openai.api_key = api_key
        # all client options can be configured just like the `OpenAI` instantiation counterpart
        openai.base_url =base_url# "http://127.0.0.1:8000/v1/"
        openai.default_headers = {"x-foo": "true"}
        
        client=openai
    
    return client

# 方法1:通过prompt，凭空构建数据集

In [38]:
key=read_key_from_file("key.txt")
# client=get_client("...","http://127.0.0.1:8000/v1/","",False)
client=get_client(key,"https://mixcopilot.openai.azure.com","gpt-35-turbo-16k",True)

model="gpt-35-turbo-16k"

def chat(messages,temperature=0.5,max_tokens=1354):
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return completion.choices[0].message.content


prompt = "A model that takes in Chinese sentences, then rewrites them in the style of Lu Xun"
temperature = .5
number_of_examples = 100

# 创建
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\nresponse_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model."
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })
    print(messages)
    messages.append({
                "role": "user",
                "content": f"Here is the type of model we want to train:\n`{prompt.strip()}`",
            })
    
    return chat(messages,temperature)


# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
[{'role': 'system', 'content': 'You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\nresponse_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.'}]
Generating example 1
[{'role': 'system', 'content': 'You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\n


KeyboardInterrupt



In [26]:
def generate_system_message(prompt):
    messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ]

    return chat(messages,temperature,500)

system_message = generate_system_message(prompt)

print(f'system prompt是: `{system_message}`. 如果不满意，请重新生成')

system prompt是: `Given a Chinese sentence, rewrite it in the style of Lu Xun and output short phrases for chatting, with added emojis.`. 如果不满意，请重新生成


In [30]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 10 successfully-generated examples. Here are the first few:


,prompt,response
0,Rewrite the following Chinese sentence in the ...,今天的天气真是宜人，阳光明媚，大地万物焕发生机。
1,Rewrite the following Chinese sentence in the ...,吾爱汝，永不分离。
2,Rewrite the following Chinese sentence in the ...,他乃是一位勇敢而坚定之人。
3,Rewrite the following Chinese sentence in the ...,此都市充盈着繁忙与喧嚣。
4,Rewrite the following Chinese sentence in the ...,他们在夜晚的街道上闲逛。


In [35]:
import json

# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Convert dataframes to dictionaries
train_dict = train_df.to_dict(orient='records')
test_dict = test_df.to_dict(orient='records')

# Save dataframes to JSON files
with open('./dataset/train.jsonl', 'w', encoding='utf-8') as train_file:
    for record in train_dict:
        json.dump(record, train_file, ensure_ascii=False, indent=4)
        train_file.write('\n')

with open('./dataset/test.jsonl', 'w', encoding='utf-8') as test_file:
    for record in test_dict:
        json.dump(record, test_file, ensure_ascii=False, indent=4)
        test_file.write('\n')

        

# train_dataset = load_dataset('json', data_files='./dataset/train.jsonl', split="train")
# valid_dataset = load_dataset('json', data_files='./dataset/test.jsonl', split="train")

# 方法2：基于已有数据集，做改写

In [6]:
key=read_key_from_file("key.txt")
# client=get_client("...","http://127.0.0.1:8000/v1/","",False)
client=get_client(key,"https://mixcopilot.openai.azure.com","gpt-35-turbo-16k",True)

model="gpt-35-turbo-16k"

# 填写本地LLM的地址,修改prompt
def parse_text(content):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "your are chinese,use chinese ", # 中文输出
            },
            {
                "role": "user",
                "content": content,
            },
        ],
    )
    
    return completion.choices[0].message.content

# 使用LLM 把数据集处理成新的数据

In [9]:
json_path="./dataset/minecraft_science_fiction_story_zh"

import hashlib
# 取唯一id，去重
def get_string_hash(string):
    md5_hash = hashlib.md5()
    md5_hash.update(string.encode('utf-8'))
    return md5_hash.hexdigest()

# 判断是否已经存在
def add_new_data(original_list, item):
    is_has=False
    for element in original_list:
        if item['id']==element["id"]:
            is_has=True
    if is_has==False:
        original_list.append(item)
    return original_list


# 判断是否已经存在
def is_new_data(original_list, item):
    is_has=True
    for element in original_list:
        if item['id']==element["id"]:
            is_has=False
    return is_has



new_dataset=[]

try:
    # 打开本地的JSON文件
    with open(json_path, 'r') as file:
        # 读取文件内容
        new_dataset = (json.load(file))["train"]
except:
    print(f"")



In [19]:

for data in dataset[0:300]['train']:
    # print(data)
    question=data['question']
    answer=data['answer']

    _id=get_string_hash(question+answer)

    # 判断是否是新数据
    if is_new_data(new_dataset, {
        "id":_id
    }):
        
        print(_id,question)
        print('')
        text=parse_text('''{0} , {1}
        结合以上文本信息，写成一个科幻小故事，200字以内的小故事。
        '''.format(question,answer))
     
        keywords=parse_text('''{0}
        提炼3个中文关键词，输出：#关键词
        '''.format(text))
        print('')
        print('------')
        print(keywords)
        print('------')
        print('')
        print(text)
        print('------')
        print('')
    
        new_dataset=add_new_data(new_dataset,{
            "id":_id,
            "question":keywords,#新数据
            "answer":text,#新数据
            # "_question":question, #保留原始的数据
            # "_answer":answer
        })
        # new_dataset.append({
        #     "id":_id,
        #     "question":keywords,#新数据
        #     "answer":text,#新数据
        #     # "_question":question, #保留原始的数据
        #     # "_answer":answer
        # })
        print(len(new_dataset))
        print('')

e0fdef2c0150ad5d703c8913b2026520 What is the primary purpose of adding a custom font to a Minecraft server's resource pack?


------
#虚拟世界 #自定义字体 #创意改变
------

诺亚是一名年轻的电脑工程师，在未来的2030年，他参与了一项名为“虚拟世界”的创新项目。这个项目是通过一个全新的虚拟现实技术，将人们的意识连接到一个虚拟世界中。人们可以在这个世界中具备自由行动能力，并与其他参与者互动。

作为项目的核心开发人员之一，诺亚负责创建一个独特的虚拟世界的资源包。他决定加入一个自定义字体来提升这个世界的体验。这个自定义字体被设计成一种神秘的符号语言，用于表示不同的能力和技能。

在虚拟世界中，每个人都可以选择一个角色，然后使用这个角色的能力和技能探索这个世界。而自定义字体则是每个角色的标志，代表了他们在虚拟世界中的独特身份和能力。

诺亚的自定义字体成为了虚拟世界的热门话题。人们争相尝试不同的字体，展示自己的个性和风格。一些玩家将字体设计成华丽而复杂的形状，展示他们的创造力和艺术才能；一些玩家选择简洁而流畅的字体，体现他们在游戏中的实力和效率。

随着时间的推移，人们逐渐意识到字体不仅仅是一种装饰，它还可以传递一种思想和意识。有些玩家开始设计自己的字体，试图通过这种方式传达一种特殊的信息或者理念。

这个自定义字体的应用远远超出了诺亚的预期。它成为了虚拟世界中的一种文化符号，代表了创造力、个性和力量。人们开始在虚拟世界中组建字体社区，分享自己的设计和创意。字体成为了人们交流和理解的一种重要方式。

正是因为诺亚的自定义字体，虚拟世界成为了一个真正多样化和创造性的地方。人们可以通过字体表达自己的个性，建立起新的连接和友谊。这个虚拟世界进一步繁荣，为人们提供了一个不同于现实世界的乐土。

这个科幻小故事告诉我们，一个简单的创意和改变，可以在某个领域产生无限的可能性和影响力。诺亚的自定义字体改变了虚拟世界的文化和交流方式，让人们享受到了一个全新的虚拟体验。
------

198

3b9d3fbe661bad6cad1e1a1c54763ae1 What is the level range where diamond ore c

In [20]:
# 保存新数据
new_dataset_to_save = {"train": new_dataset}

def save_json_to_file(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

save_json_to_file(new_dataset_to_save,"./dataset/minecraft_science_fiction_story_zh")

请在此处写下您的提示。尽可能详细地描述！
然后，选择生成数据时要使用的温度（介于0和1之间）。较低的值非常适合精确的任务，如编写代码，而较大的值则更适合创造性的任务，如编写故事。
最后，选择要生成的示例数量。生成的示例数量越多，a) 生成时间越长，b) 数据生成的成本越高。但一般来说，生成更多的示例会导致更高质量的模型。通常，最小值为100。


"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"